In [1]:

from pathlib import Path
import numpy as np
import random
from collections import Counter
from omnibelt import load_json


In [3]:

root = Path('../data/backup')
path = root / 'cladder-v1-q-aggregate.json'
path = root / 'cladder-v1-q-commonsense.json'
path = root / 'cladder-v1-q-balanced.json'
# path = '../data/cladder-v1-common-easy.json'
full = load_json(path)
full_ids = {entry['question_id']: entry for entry in full}
models = load_json(root / 'cladder-v1-meta-models.json')
model_table = {info['model_id']: info for info in models}
len(full), len(models)

(10112, 7064)

In [4]:
# models[1]

In [37]:
full[8003]

{'question_id': 21241,
 'desc_id': 'obesity_mortality-arrowhead-backadj-model4576-spec12-q1',
 'given_info': 'Method 1: We look directly at how obesity correlates with lifespan in general. Method 2: We look at this correlation case by case according to diabetes.',
 'question': 'To understand how obesity affects lifespan, is it more correct to use the Method 1 than Method 2?',
 'answer': 'yes',
 'meta': {'story_id': 'obesity_mortality',
  'graph_id': 'arrowhead',
  'flipped': False,
  'polarity': True,
  'groundtruth': [],
  'bad_candidate_set': ['V3'],
  'given_info': [[], ['V3']],
  'query_type': 'backadj',
  'rung': 2,
  'formal_form': '[backdoor adjustment set for Y given X]',
  'treatment': 'X',
  'outcome': 'Y',
  'model_id': 4576},
 'reasoning': None}

In [5]:
key = 'rung'
vocabs = {}
for entry in full:
	vocabs.setdefault(entry['meta'][key], []).extend(entry['question'].split())
	vocabs.setdefault(entry['meta'][key], []).extend(entry['given_info'].split())
	model = models[entry['meta']['model_id']]
	vocabs.setdefault(entry['meta'][key], []).extend(model['background'].split())
def _cleanup(w):
	w = w.lower()
	w = w.replace('?', '').replace('.', '').replace(',', '').replace('\'', '').replace('\"', '').replace('%', '').replace(':', '')
	# check if it's a number
	try:
		float(w)
		return 'NUM'
	except:
		pass
	return w
vocabs = {k: [_cleanup(w) for w in v] for k, v in vocabs.items()}
vocabs = {k: Counter(v) for k, v in vocabs.items()}
vocabs.keys()

dict_keys([3, 2, 1])

In [6]:
# vocabs

In [7]:
{k: len(v) for k, v in vocabs.items()}

{3: 479, 2: 400, 1: 405}

In [8]:
random.seed(0)
indices = random.choices(list(range(len(full))), k=50)
print(indices)
assert len(set(indices)) == len(indices)

[8538, 7664, 4252, 2618, 5170, 4094, 7925, 3067, 4819, 5899, 9182, 5103, 2849, 7642, 6252, 2533, 9199, 9937, 8192, 9122, 3136, 7380, 9089, 6916, 4774, 1018, 4390, 6177, 9232, 9774, 4823, 8750, 2634, 8140, 5548, 141, 7277, 4032, 8340, 6756, 11, 4991, 8773, 2466, 3288, 8802, 1932, 5738, 2412, 9783]


In [9]:
for i, ind in enumerate(indices):
	entry = full[ind]
	model = models[entry['meta']['model_id']]
	print(i)
	# print(model['background'])
	# print(entry['given_info'])
	# print(entry['question'])
	print(entry['answer'])
	print()

0
no

1
yes

2
yes

3
yes

4
yes

5
yes

6
no

7
yes

8
no

9
no

10
yes

11
no

12
no

13
yes

14
no

15
yes

16
yes

17
yes

18
no

19
no

20
no

21
no

22
yes

23
no

24
no

25
yes

26
no

27
no

28
no

29
no

30
no

31
no

32
yes

33
yes

34
yes

35
yes

36
no

37
yes

38
yes

39
yes

40
no

41
yes

42
no

43
no

44
no

45
no

46
yes

47
yes

48
no

49
yes



In [10]:
len([e for e in full if e['desc_id'].startswith('nonse')])/len(full)

0.37994462025316456